In [ ]:
import pyvisa
import time
import numpy as np
import pandas as pd

from device import Scope
from device import Stage

In [ ]:
rm = pyvisa.ResourceManager()
visa_list = rm.list_resources()
visa_list

In [ ]:

# stage = stage.Autostage(visa_list[0])#三軸の接続先設定
# scope = tek7k.Scope(visa_list[1])#オシロスコープの接続先指定
scope = rm.open_resource(visa_list[1])
func = rm.open_resource(visa_list[2])



In [ ]:
#初期条件
scope.timeout = 30000 # timeout の設定# 30 秒
scope.average(10000)#averaging回数指定

In [ ]:
# Setting source as Channel 1
scope.write('DATA:SOU CH1') 
scope.write('DATA:WIDTH 2') 
scope.write('DATA:ENC SRIBINARY')

scope.write('DATa:STARt 1')
scope.write('DATa:STOP 5000')
ymult = float(scope.query('WFMOUTPRE:YMULT?')) # y-axis least 
yzero = float(scope.query('WFMOUTPRE:YZERO?')) # y-axis zero error
yoff = float(scope.query('WFMOUTPRE:YOFF?'))   # y-axis offset
xincr = float(scope.query('WFMOUTPRE:XINCR?')) # x-axis least count
xoff = float(scope.query('WFMP:PT_OFF?'))   # x-axis offset

df = pd.DataFrame()

In [ ]:
# FREQuency {<frequency>|MINimum|MAXimum}
for f in range(500,2500,25): #500KHz 2500KHz
    func.write('FREQuency '+ f*1000) 
    time.sleep(5)#5秒止める

    ADC_wave = scope.query_binary_values('CURVe?',datatype = 'h',is_big_endian=False, container=np.array)
    Volts = (ADC_wave - yoff) * ymult + yzero
    df[f] = Volts
    # plt.plot(Volts) # データ

df["time"] = time
time = np.arange(-1*xincr*xoff,-1*xincr*xoff+len(Volts)*xincr,xincr)

In [ ]:
df
df.to_csv("f.csv")